# Testing Revolve checkpointing: Burger's equation

This notebook aims to execute tests to verify the pyadjoint with checkpointing implementation.

In the current case, we consider Burger's equation, which is a non-linear equation for the advection and
diffusion of momentum. Here we choose to write the Burgers equation in
one dimension:
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} - \nu \frac{\partial^2 u}{\partial x^2} = 0.
$$

Additional information about time and spatial discretisation is available in the notebook.

As usual, we begin by importing Firedrake.

In [1]:
from firedrake import *

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


We now import the `firedrake.adjoint` package, which computes adjoint-based gradient automatically.

In [2]:
from firedrake.adjoint import *

The use of adjoint calculations to compute the gradient of a quantity of interest resulting from the solution of a system of partial differential equations (PDEs) is widespread and well-established. The resulting gradient may be employed for many purposes, eg, for inverse problems.

Solving the adjoint to a non-linear time-dependent PDE requires the forward PDE to be solved first. The adjoint PDE is then solved in a reverse time order, but depends on the forward state. Storing the entire forward state in preparation for the adjoint calculation has a memory footprint linear in the number of time steps. In contrast, checkpointing approaches store only the state required to restart the forward calculation from a limited set of steps. As the adjoint calculation progresses, the forward computation is progressively rerun from the latest available stored state up to the current adjoint step. This enables less forward state to be stored, at the expense of a higher computational cost as forward steps are run more than once. 

[Griewank and Walther (2000)](https://dl.acm.org/doi/pdf/10.1145/347837.347846) proposed a checkpointing algorithm referred to as Revolve that is available in the Python package [checkpoint_schedules](https://www.firedrakeproject.org/checkpoint_schedules/). 

[checkpoint_schedules](https://www.firedrakeproject.org/checkpoint_schedules/) provides schedules for step-based incremental checkpointing of the adjoints to computer models. The schedules contain instructions indicating the sequence of forward and adjoint steps to be executed and the data storage and retrieval to be performed. checkpoint_schedules can be installed using `pip`: 

`pip install checkpoint-schedules`

To employ Revolve checkpointing in your adjoint-based gradient, you would install checkpoint_schedules and import the Revolve algorithms as shown below.

In [3]:
from checkpoint_schedules import Revolve

On employing `firedrake.adjoint`, we need to start taping (annotating) the Firedrake operations we run in order to be able to execute the adjoint.

In [4]:
continue_annotation()

True

We can now proceed to set up the problem. We choose a resolution and set up a unit interval mesh and degree 2 continuous Lagrange polynomials.

In [5]:
n = 100
mesh = UnitIntervalMesh(n)
V = FunctionSpace(mesh, "CG", 2)

Next, we choose the end step and the time-step adopted in the texts.

In [6]:
end = 0.05
timestep = 1.0/n
steps = int(end/float(timestep)) + 1

We are employing backward Euler.

In [7]:
def Dt(u, u_, timestep):
    return (u - u_)/timestep

We then write a function to compute the cost functional which is given by the expression:
$$
\min_{J, u, u_0} \ \int_{\Omega} u \cdot \nabla u\ + u_0 \cdot \nabla u_0\,\text{d}x 
$$
where $u_0$ is the initial condition $u_0 = u(x, 0) = sin (2\pi x)$. The objective is minimising the cost function subject to the burger's equation and Dirichlet conditions $u(0, t) = u(1, t) = 0.0$.

Below, the function `J` computes the forward solver, i.e., Burger's equation and the cost function. The arguments of `J` are the initial condition `ic`, the solver type `solve_type` and the checkpointing that is a `bool` type. 

Within `J`, we set the `u_` and `u` (lines 2 and 3) which are the solution functions for the current and the next timestep. The cost function is computed at the line 30. 

The time advancing is executed with a `for` loop. If `checkpointing=True`, we run the  `tape.timestepper` iterator that advances the tape timestep. 


In [8]:
def J(ic, checkpointing):
    u_ = Function(V)
    u = Function(V)
    v = TestFunction(V)
    u_.assign(ic)
    nu = 0.0001
    F = (Dt(u, u_, timestep)*v
         + u*u.dx(0)*v + nu*u.dx(0)*v.dx(0))*dx
    bc = DirichletBC(V, 0.0, "on_boundary")

    problem = NonlinearVariationalProblem(F, u, bcs=bc)
    solver = NonlinearVariationalSolver(problem)

    def time_advance():
        solver.solve()
        u_.assign(u)
    tape = get_working_tape()
    t = 0
    if checkpointing:
        for _ in tape.timestepper(np.arange(t, end + float(timestep), float(timestep))):
            time_advance()
    else:
        for _ in np.arange(t, end + float(timestep), float(timestep)):
            time_advance()

    return assemble(u_*u_*dx + ic*ic*dx)

We write a function to execute tests with the burger's equation when computing the automated gradient with the checkpointing algorithms. We need to inform the adjoint solver to employ the checkpointing algorithm in the gradient computations. That is achieved with code line 7, i.e., with the code `tape.enable_checkpointing(Revolve(steps, 5))`. In this case, we are informing the code to use Revolve algorithmic, where the forward data stored in memory is only for 5 steps.


In [9]:
def test_burgers_newton():
    """
    Adjoint-based gradient tests with and without checkpointing.
    """
    tape = get_working_tape()
    tape.progress_bar = ProgressBar
    tape.enable_checkpointing(Revolve(steps, 5))
    x, = SpatialCoordinate(mesh)
    ic = project(sin(2.*pi*x), V)
    val = J(ic, solve_type, True)
    if checkpointing:
        assert len(tape.timesteps) == steps

    Jhat = ReducedFunctional(val, Control(ic))
    dJ = Jhat.derivative()
    # Test recompute forward model
    assert(np.allclose(Jhat(ic), val))

    dJbar = Jhat.derivative()
    # Test recompute adjoint-based gradient
    assert np.allclose(dJ.dat.data_ro[:], dJbar.dat.data_ro[:])

    # Taylor test
    h = Function(V)
    h.assign(1, annotate=False)
    assert taylor_test(Jhat, ic, h) > 1.9
    print("Taylor tests passed!")

Additionally, we compare the adjoint-based gradient results with and without checkpointing witht the function `test_checkpointing_validity`.

In [10]:
def test_checkpointing_validity():
    """Compare forward and backward results with and without checkpointing.
    """
    # Without checkpointing
    tape = get_working_tape()
    x, = SpatialCoordinate(mesh)
    ic = project(sin(2.*pi*x), V)

    val0 = J(ic, solve_type, False)
    Jhat = ReducedFunctional(val0, Control(ic))
    dJ0 = Jhat.derivative()
    val00 = Jhat(project(sin(1.*pi*x), V))
    dJ00 = Jhat.derivative()
    tape.clear_tape()
    print("Without checkpointing")
    # With checkpointing
    tape.progress_bar = ProgressBar
    tape.enable_checkpointing(Revolve(steps, 1))
    x, = SpatialCoordinate(mesh)
    ic = project(sin(2.*pi*x), V)
    val1 = J(ic, solve_type, True)
    tape.visualise("tape_before_recompute.pdf")
    Jhat = ReducedFunctional(val1, Control(ic))
    dJ1 = Jhat.derivative()
    val11 = Jhat(project(sin(1.*pi*x), V))

    tape.visualise("tape_after_recompute.pdf")
    dJ11 = Jhat.derivative()
    tape.visualise("tape_after_gradient.pdf")
    assert len(tape.timesteps) == steps
    assert np.allclose(val0, val1)
    assert np.allclose(dJ0.dat.data_ro[:], dJ1.dat.data_ro[:])
    print("Comparisons between automated gradient with and without checkpoint are working as expected!")

In [11]:
test_checkpointing_validity()
tape = get_working_tape()
tape.clear_tape()

Taping forward ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]

Without checkpointing



Evaluating Adjoint ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 21/21 [0:00:00]
Evaluating Functional ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]
Evaluating Adjoint ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 21/21 [0:00:00]


Comparisons between automated gradient with and without checkpoint are working as expected!


In [13]:
test_burgers_newton()
tape.clear_tape()

Taping forward ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]
Evaluating Adjoint ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 11/11 [0:00:00]
Evaluating Functional ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]
Evaluating Adjoint ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 11/11 [0:00:00]
Evaluating Functional ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]
Evaluating Adjoint ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 11/11 [0:00:00]
Evaluating Functional ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]
Evaluating Functional ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]
Evaluating Functional ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]
Evaluating Functional ▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣ 6/6 [0:00:00]

Running Taylor test
Computed residuals: [0.0001981937668922317, 4.9548558052393515e-05, 1.2387146782200635e-05, 3.0967871499368214e-06]
Computed convergence rates: [1.9999966128589979, 1.9999991533885053, 1.9999997883156315]
Taylor tests passed!


The tests above are working as expected! Now, let us consider another time dependent optimisation problem with checkpointing approach. The next notebook is simplified implementation of a Full Inversion problem.  